<a href="https://colab.research.google.com/github/yonathanarbel/AI-LAW/blob/main/Class_4_Working_with_an_LLM_Class_Excerise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab file will take us through the steps necessary to speak with an LLM Programtically.

By programtically, I mean that rather than accessing the website, we let Python talk to the LLM. This opens up a host of exciting use cases and allows us to BUILD applications with LLMs.

The first step on our journey is to get an API key. Think of it as an access password.
(we are not, just yet, running LLMs on our own computers. We are using services of cloud providers today, specifically, Groq)





How do we get our API key?
https://console.groq.com/login


Create aPI key

Give it a name like "AI-for-lawyers"



copy the key and put it in the cell below

Please remember that the API key is linked to your account, so don't let others access it, and make sure it doesn't find its way to the internet. You can save it to your computer, and remove it from the notebook when you are done.



In [1]:
#critical, add your key

API_KEY = ""



In [2]:
#now, some installs & definitions
!pip install groq
import textwrap
from IPython.display import Markdown, display
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from groq import Groq
import requests
from bs4 import BeautifulSoup
import textwrap
from IPython.display import display, Markdown
import base64
import requests
from groq import Groq



#this is where we tell groq our API key
client = Groq(
    api_key=API_KEY,
)

#I also include one function just to make things easier on the eye.

def pretty_print(text, width=80):
    # Split the text into paragraphs
    paragraphs = text.split('\n\n')

    # Wrap each paragraph
    wrapped_paragraphs = []
    for p in paragraphs:
        # Wrap each paragraph while respecting the breaks
        wrapped_paragraphs.append(textwrap.fill(p.strip(), width=width))

    # Join the wrapped paragraphs
    formatted_text = '\n\n'.join(wrapped_paragraphs)

    # Display as Markdown
    display(Markdown(formatted_text))



In [3]:
#we now have access to a few models. Which?

url = "https://api.groq.com/openai/v1/models"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

# Extract the necessary fields and convert to a pandas DataFrame
models_data = response.json()['data']
models_table = pd.DataFrame(models_data)[['id', 'owned_by', 'context_window']]

# Display the table
print(models_table)

                                       id      owned_by  context_window
0              llava-v1.5-7b-4096-preview         Other            4096
1                      mixtral-8x7b-32768    Mistral AI           32768
2                        llama-guard-3-8b          Meta            8192
3                            gemma2-9b-it        Google            8192
4   llama3-groq-70b-8192-tool-use-preview          Groq            8192
5                 llama-3.1-70b-versatile          Meta          131072
6              distil-whisper-large-v3-en  Hugging Face             448
7                        whisper-large-v3        OpenAI             448
8                    llama-3.1-8b-instant          Meta          131072
9                             gemma-7b-it        Google            8192
10                        llama3-70b-8192          Meta            8192
11   llama3-groq-8b-8192-tool-use-preview          Groq            8192
12                         llama3-8b-8192          Meta         

In [4]:
#choosing the model
#We are starting with language models, so let's pick one
model_id = "llama-3.1-70b-versatile"



In [5]:
#define a query function
# Define a query function to send the audio file
def prompt_llm(prompt="", model_id="whisper-large-v3", image_url=None, audio_file_path=None):
    content = [{"type": "text", "text": prompt}]

    if image_url:
        content.append({
            "type": "image_url",
            "image_url": {"url": image_url}
        })

    # If an audio file is provided, we will send it using the audio API
    if audio_file_path:
        with open(audio_file_path, "rb") as audio_file:
            # Here we use the audio transcription API directly
            transcription = client.audio.transcriptions.create(
                file=("audio_file.mp3", audio_file),  # Send the file as binary
                model=model_id,
                response_format="text"  # You can change this to "json" or "verbose_json"
            )
            return transcription  # Return the transcription text directly

    # If no audio file, we send the normal text or image request
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": content,
            }
        ],
        model=model_id,
    )
    return chat_completion.choices[0].message.content




In [6]:
#let's have fun

answer = prompt_llm(prompt = "In what ways is the study of torts inferior to the study of contracts?", model_id = model_id)
pretty_print(answer)

The study of torts is not necessarily inferior to the study of contracts, as
both are essential areas of the law. However, some arguments can be made that
tort law has some limitations and challenges that may make it seem less
systematic and less comprehensive compared to contract law:

1. **Lack of clear boundaries and definitions**: Torts is a broader and more
amorphous area of the law. It encompasses a wide range of actionable wrongs,
including intentional torts (e.g., assault, battery), negligent torts (e.g.,
personal injury, property damage), and strict liability torts (e.g., product
liability). This diversity of wrongful conduct makes it more challenging to
define and categorize torts.

2. **Limited guidance from precedents**: Tort cases often involve unique fact
patterns, which can make it more difficult to discern clear precedents and rules
of law. Judges and scholars must often rely on analogous cases or resort to
policy-based reasoning to resolve disputes. This lack of guidance from
precedents can make it more challenging to teach and apply tort law.

3. **Complexity of establishing duty, breach, causation, and damages**: Tort
claims require plaintiffs to prove four essential elements: duty (or
obligation), breach of duty, causation, and damages. Each element involves a
distinct inquiry and requires a nuanced understanding of the relevant case law,
statutes, and policy considerations. Contract law, by contrast, typically
emphasizes the contractual terms and conditions that govern the relationship
between the parties.

4. **Limited opportunity for theoretical coherence**: Tort law may be less
amenable to theoretical coherence due to its focus on resolving factual disputes
and addressing various forms of wrongdoing. Contract law, by contrast, can be
understood through a more theoretical framework (e.g., formalism, laissez-faire
economics), which can provide greater structure and cohesion to the subject.

5. **Frequent reliance on public policy**: Tort courts often rely on public
policy considerations to shape the development of tort law. While this may allow
courts to address novel issues and ensure tort law adapts to changing societal
conditions, it also creates controversy and introduces uncertainty. Contract law
typically provides more deference to private orderings and market arrangements.

6. **Limited emphasis on consent and agreement**: Tort law is primarily
concerned with harm suffered by the plaintiff and whether the defendant is
responsible for that harm. In contrast, contract law emphasizes the parties'
mutual consent and agreement as the foundation of contractual obligations. Some
argue that this focus on consent and agreement provides greater structure and
coherence to contract law.

Ultimately, both torts and contracts are crucial areas of the law that require
attention and understanding. The perceived limitations and challenges of tort
law should not imply its inferiority but rather highlight its complexities and
nuances.

In [7]:
#Ok, now that we had fun, let's do some more serious stuff







### Scenario 1: Li Wei - Data Retention Concern

Li Wei is a 42-year-old software engineer living in Shanghai, China. In 2010, she purchased her first iPhone while on a business trip to the United States. She created an Apple ID to set up the device and has been using various Apple services since then.

Recently, Li Wei became concerned about data privacy and decided to request all the data Apple had stored about her. To her surprise, she discovered that Apple still retained detailed information about her initial iPhone purchase from 14 years ago, including the transaction details and her shipping address at the time.

Li Wei is worried that Apple might be violating its own data retention policies by keeping this information for so long. She wants to know if she has any recourse under Apple's privacy policy or applicable laws.

### Scenario 2: Akira Tanaka - Cross-Border Data Transfer

Akira Tanaka is a 28-year-old graphic designer from Tokyo, Japan. He's been using Apple products for years and recently started using iCloud+ for storing his work files. Akira frequently travels between Japan and the United States for client meetings.

During a recent trip to New York, Akira noticed that he could access all his iCloud data seamlessly, just as he could in Japan. This made him wonder about the implications of cross-border data transfers. He's particularly concerned about whether his professional work, which sometimes includes confidential client information, is adequately protected when he accesses it from different countries.

Akira wants to understand Apple's policies on international data transfers and storage. He's especially interested in knowing where his data is physically stored, how it's protected during transfers, and whether there are any specific considerations he should be aware of as a user who frequently works across international borders.

In [8]:
#Consider Apple's privacy policy for the Apple ID

url = "https://www.apple.com/legal/privacy/data/en/apple-id/"

def fetch_policy(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    content = soup.find('main', class_='main')
    if content:
        # Preserve the breaks between sections and paragraphs
        return content.get_text(separator='\n\n', strip=True)
    else:
        return "Couldn't find the main content."

policy = fetch_policy(url)

pretty_print(policy)

English

Choose your preferred language

Apple ID & Privacy

Your Apple ID information is used to enable Apple services when you sign in,
including iCloud Backup, which automatically backs up the data on your device in
case you need to replace or restore it. Your device serial number may be used to
check eligibility for service offers.

Apple ID is designed to protect your information and enable you to choose what
you share.

When you sign in with your Apple ID, you will enable services such as iCloud,
the App Store, iMessage, and Find My on this iPhone.

Certain data, including your contacts, calendars, photos, documents, health,
activity, and other app data, will be sent to Apple to store and back up on your
behalf.

In addition, your device will be associated with your Apple ID to provide you
with better service and support.

Your Apple ID and all Apple services are designed from the ground up to protect
your privacy. We work hard to collect only the data we need to make your
experience better. When we do collect data, we believe it’s important for you to
know what we’re collecting and why we need it, so you can make informed choices.

An Apple ID is the personal account you use to access Apple services like
iCloud, the App Store and other Apple online stores, iMessage, and FaceTime, and
to access your content across all your devices and the web. Your Apple ID
includes the information you use to sign in, as well as all the contact,
payment, and security details, such as trusted phone numbers or security
questions, that you use across Apple services. Apple retains your device’s name,
serial number, and other hardware identifiers to provide you with better service
and support, including as a convenience to display the warranty status of your
device and to check your eligibility for service offers when you activate a
device. You can edit your Apple ID information and view a list of your devices
and phone numbers on your Apple ID account page at

appleid.apple.com

.

When you sign in with your Apple ID, you will enable services such as iCloud,
the App Store, iMessage, FaceTime, and Find My on this iPhone. Your Apple ID
account information, including your associated phone number, will be used with
each service, which will generate analytics information that may be used to
maintain and improve those services. Additionally, certain data from your
device, including your selected profile photo, your contacts, calendars, photos,
documents, health, activity, Safari tabs, and other app data, will be sent to
Apple to store and back up on your behalf.

Apple collects information associated with your Apple ID to determine whether
your device is eligible for certain legally required features and services,
which are based on the country or region in which your device is located. We
also collect information about the apps you install, including from where you
install them (for example, a website or app marketplace), associated with your
Apple ID for accounting, reporting, analytics, and fraud detection purposes in
connection with the enablement of these legally required features and services.
In addition, we use the age associated with your Apple ID to comply with our
obligations, including providing age-appropriate information about your privacy
rights in certain cases.

iCloud and Privacy

Signing in to iCloud

When you use iCloud, certain data you store on your device will be automatically
sent to and stored by Apple. This allows you to access your data on all your
iCloud-enabled devices or computers (“devices”) automatically. This data may
include:

Contacts

Calendars

Reminders

Bookmarks

Safari tabs

Health data

Home data

Notes

Freeform

Photos

Documents

Wallet data

Keychain and passwords

Phone number on your device

Device and account settings

Data from other Apple apps that use iCloud

Data from third-party apps that use iCloud

iCloud Backup is also enabled to help ensure you have a current backup of the
data on your device in case you need to replace or restore it.

You can adjust which iCloud features are enabled and what data is sent to iCloud
from Settings on your device at any time.

iCloud Data Security

iCloud protects your information by encrypting it when it’s in transit, storing
it in iCloud in an encrypted format, and using secure tokens for authentication.
For certain information, Apple uses end-to-end encryption. This means that, by
design, only you can access this information, and only on devices where you’re
signed in to iCloud. No one else, not even Apple, can access end-to-end
encrypted information. For more information on iCloud Data Security, visit

support.apple.com/102651

.

Both Apple and third-party data centers may be used to store and process your
data. When processing data stored in a third-party data center, encryption keys
are accessed only by Apple software running on secure servers, and only while
conducting the necessary processing.

iCloud Keychain

iCloud Keychain keeps your Safari website usernames and passwords, credit card
information, and Wi-Fi network information up to date and available across all
your devices. It can also store the account information you use for Mail,
Contacts, Calendar, Messages, Facebook, Twitter, LinkedIn, and other internet
accounts and automatically add this information to all of your devices.

iCloud Keychain uses end-to-end encryption and is designed so that only you can
access this information and only on devices where you’re signed in to iCloud. No
one else, not even Apple, can access your iCloud keychain data. You can choose
to turn off iCloud Keychain in your iCloud settings at any time.

iCloud Backup

When you use iCloud Backup, your backup may include photos, videos, messages
(iMessage, SMS, and MMS), Health and other app data, location settings (such as
location-based reminders that you have set up), Visual Voicemail, device
settings, device characteristics, and other data from your iOS or iPadOS device
depending on your Settings.

You can stop iCloud from backing up your app data for a particular app by going
to Settings > [your name] > iCloud > Manage Storage > Backups, selecting your
iOS or iPadOS device in the Backups section, and tapping to turn off the apps
for which you’d like to disable backup.

You can turn off iCloud Backup entirely from iCloud settings and use Finder
(macOS Catalina 10.15 or later) or iTunes (macOS Mojave 10.14 or earlier) to
back up your iOS or iPadOS device to your computer. To completely remove your
backup data from iCloud, you can delete your individual backups before turning
off iCloud Backup. Otherwise, the backup data will be retained for up to 180
days.

Sharing Data with iCloud

If you use iCloud to share files either publicly or privately in Notes, Pages,
Numbers, Keynote, Freeform, and certain third-party apps, Apple will store and
have access to the shared file. Your first and last name associated with your
account (if applicable), the phone number on your device, and the file name will
be available to anyone who has access to the sharing link, regardless of whether
your share is private or public. The identities of private share participants
will not be made available to anyone who has not been invited to and accepted
the private share.

If you create or join a Shared Library, all participants can add, edit, or
delete photos and videos in the Shared Library through iCloud. There are several
ways to add content to a Shared Library either manually or with automatic
sharing. You can control how you want to share content by going to Settings >
Photos > Shared Library > Share from Camera on your iOS or iPadOS device.

iCloud and Apps

Some apps use iCloud to store data and content you provide to the app, so you
can access the data across all your iCloud-enabled devices. The app may choose
to store some or all of your data in iCloud. Third-party apps can request to
look you up, so that other users of that application can see your first and last
name when they search by Apple ID.

You can stop an app from storing data in iCloud by turning off its access to
iCloud. To do so, go to Settings > [your name] > iCloud, and tap to turn off
apps shown in the list.

Apple Services That Require iCloud

To use Apple services like Apple Pay, Activity sharing, Home, Handoff, Universal
Clipboard, and Unlock with Apple Watch, you must be signed in to iCloud on your
device. Apple uses your iCloud plan to verify your eligibility to set up and use
Secure Camera Recording in the Home app. Apple can’t access the content of your
camera recordings. You do not need to use all iCloud features and can turn off
any iCloud features you don’t want to use in iCloud settings on your device.

Signing Out of iCloud

If you want to stop using iCloud on your iOS or iPadOS device and any paired
Apple Watch, you can sign out of iCloud by going to Settings > [your name] and
tapping Sign Out. To stop using iCloud on your Mac, go to System Settings >
[your name] and click Sign Out. Your Apple ID password may be required.

Communicating with You

Apple may use information about your account, such as the Apple products you
own, devices registered to your account, and your subscriptions to Apple
services, to send you communications about iCloud and other Apple products,
services, and offers that may be of interest to you, including Apple One. Your
device serial number may be used to check eligibility for service offers. If you
are in a Family Sharing group, Apple may send you communications about products,
services, and offers available to you through Family Sharing. If you purchase an
Apple One subscription, we may send you communications about the features of
each of the services for which you have subscribed.

Email Marketing

You may receive these communications by email, if you consent to receive email
marketing. You can change your email preferences and opt out of receiving these
emails by going to

appleid.apple.com

, or on your iOS or iPadOS device by going to Settings > [your name] > Personal
Information.

Push Notifications

You may also receive these communications by push notification. To update your
notification preferences or to opt out of notifications on your iOS or iPadOS
device, go to Settings > Notifications.

Family Sharing

Family Sharing lets you and your family group share subscriptions, purchases,
and location. The family organizer can set up parental controls such as Ask to
Buy and Screen Time for children in the family group. For more information on
Family Sharing, visit

www.apple.com/legal/privacy/data/en/family-sharing

.

iMessage, FaceTime, and iCloud

When you sign in with your Apple ID on your iPhone, you will be signed in to
iMessage, FaceTime, iCloud, and other Apple services automatically. The phone
number on your device will also be verified and associated with your Apple ID.
Your Apple ID or phone numbers will be shown to the people you contact or share
data with, and people can reach you using your Apple ID, and email addresses or
phone numbers on your account. You can choose how people can reach you by going
to Settings > [your name] > Personal Information. You can also learn more about
iMessage and FaceTime and Privacy by going to Settings > Messages and tapping
About iMessage and FaceTime & Privacy.

iTunes and App Store and Music

When you sign in with your Apple ID on your iPhone, you will be signed in to the
iTunes Store, the App Store, and Apple Music automatically so that you can make
purchases or begin using Apple Music quickly and easily. Any purchases you
choose to make will be recorded in our systems so you can redownload them at any
time. If you use Apple Music, your listening history may be retained and used to
help provide personalized recommendations. You can choose not to share your
listening history from Apple Music settings.

If iTunes Match is available in your region and you use iTunes Match, iCloud
will collect information about media in your iTunes library, and either match
media to songs currently available in the iTunes Store or upload songs that
cannot be matched, after which those songs can be accessed by your eligible
devices.

Apple One

If your mobile network provider offers Apple One subscriptions and free trials,
Apple may check your phone number to determine if you are eligible through a
mobile network provider partner. If you signed up through your mobile network
provider, your phone number is used to identify your account and to let the
mobile network provider know that you have activated your subscription. We will
use the phone number associated with your subscription to verify your account at
sign in and to connect your Apple One subscription with your account.

Payment Methods, Gift Cards, and Adding Funds to Apple ID Balance

To make purchases from App Store, iTunes, Music, and more, you must add a valid
payment method to your Apple ID. To add a payment method on iOS, go to Settings
> [your name] > Payment & Shipping, tap Add Payment Method, and add relevant
payment details. You may add payment details by manually entering information or
taking a picture of your payment card. If you choose to add payment details by
taking a picture of your payment card, your device will pull the relevant
information from the image to populate the required fields; the image will not
be seen or stored by Apple. Apple collects and stores the payment method details
that you provide, including account number and expiration date, to carry out
your purchases, comply with law, and to prevent fraud. You may also provide
Apple with a billing and shipping address, which may be used for marketing and
analytics purposes, as well as to comply with law and prevent fraud.

When you use a payment card, Apple may retain and automatically update your card
number and billing information for future purchases, recurring transactions, or
other uses you authorize. Apple may obtain this information from your financial
institution or payment network, and also use it for fraud prevention and
verification.

If you have enabled Apple Pay, Apple will also check whether any Apple Pay
payment methods are eligible to be added as a payment method on file to your
Apple ID and display those payment methods below Found in Wallet. When you
conduct a transaction with your Apple ID, Apple may also check whether any
Apple ID payment methods on file are eligible for Apple Pay. If so, to help
safeguard your payment information, Apple may enable Apple Pay for that payment
method. You can modify your payment methods at any time by going to Settings >
[your name] > Payment & Shipping.

You may purchase an Apple gift card using a payment method associated with your
Apple ID and share the gift card with a recipient by providing Apple with their
email address, your name, and the gift card amount. You may also enter a message
if you so choose. Apple will use recipient information for purposes of gift card
sales and redemptions, to comply with law, and to prevent fraud.

You may also redeem an Apple gift card by applying the gift card value to your
Apple ID balance. When you redeem an Apple gift card, information about the gift
card will be associated with your Apple ID balance and may be used by Apple to
comply with law and to prevent fraud. Certain information about your gift card
redemption, such as the number of gift cards that you have redeemed, may also be
used to personalize your redemption experience and communications from Apple to
you. You may opt out of use of gift card information for personalization by
going to the iTunes Store, scrolling to the bottom, tapping Apple ID > View
Apple ID and tapping to turn off Personalized Recommendations.

You may add funds directly to your Apple ID balance using your payment method on
file. When you add funds to your Apple ID balance, you may have the option to
set automatic amount or time intervals for recurring funds increases. When you
set up automatic funds increases, you will receive communications from Apple on
your device or by email concerning your balance or recurring funds increases
(for example confirmation notices, receipts, and reminders once you’ve
enrolled). You can update or stop the automatic funds increases at any time in
your Apple ID account settings.

After you redeem a gift card or add funds to your Apple ID balance, your updated
Apple account balance will be visible in Wallet so that you can easily use your
Apple account balance for Apple Store purchases.

To improve your experience with payment methods, gift cards, and adding funds to
your Apple ID balance, Apple may collect certain data about your interactions,
including your clicks and taps, with your Apple ID account payment settings.

Subscription Management

You can view and manage your subscriptions to Apple services and third-party
services purchased through Apple’s in-app purchase system in your Apple ID
account Settings. On iOS, go to Settings > [your name] and tap Subscriptions. To
view the details of, manage, and cancel an active or recently active
subscription, tap the subscription. You may also manage the sharing of your
subscription with your Family if you are in an iCloud family, as well as your
Renewal Receipt preferences. To improve your experience with subscription
management, Apple may collect data about your interactions, including what you
view, click, and tap, within your Apple ID account Subscription settings.

Usage Data

We also collect data on your use of iCloud to provide and improve the quality of
the service. To help address customer issues if they arise, some of this data
may be associated with your Apple ID. We may use your IP address to determine
the city and country from which you are accessing the service, but we do not
retain it.

Applicable Legal Basis for Processing Personal Data

We process your Apple ID and the associated information described above
generally for performance of your contract with Apple, as necessary for
providing the service, and to comply with our legal obligations. Where consent
is the appropriate legal basis, we seek it in accordance with applicable local
law.

Where applicable law provides, we process the following categories of personal
data as necessary for purposes of our legitimate interests or those of a third
party, including information about:

Your Apple ID account, including information you use to sign in, as well as all
the contact, payment, and security details, such as trusted phone numbers or
security questions, that you use across Apple services

The apps you install, including from where you install them (for example, a
website or app marketplace)

Whether your device is eligible for certain legally required services and
features based on the country or region in which it is located

Your devices, including your device’s name, serial number, and other hardware
identifiers

Your use of, and subscriptions to, various Apple services, such as iCloud, the
iTunes Store, the App Store, Apple Music, and Apple One

Payment details, including account numbers and expiration dates, billing and
shipping addresses, and gift card redemption information

These legitimate interests include:

Providing you and others with better service and support, and for accounting,
reporting, and analytics purposes

Determining the availability of and enabling legally required features and
services

Using your data to send you offers via email and push notifications that may be
of interest to you

Checking eligibility for offers

Preventing fraud and other malicious activity

Retention

Where you make a purchase such as a subscription, we retain personal data
associated with your purchase for the periods specified by applicable laws
relating to financial reporting, which vary by region. For most customers, that
requires at least a 10-year retention period, but in regions such as China that
period can be 30 years. We may also retain a record of your interactions with
iCloud-related services. These records are typically stored for 30 days but may
be stored for up to three years in an aggregated form, unless we are required by
law or legal process to store them for a longer period.

If you delete your Apple ID account, we retain a record of the deleted account
to comply with legal obligations, including as described above, and for so long
as necessary for the legitimate interests of account security. If you choose to
create a new account later, you will need to use an email address that is not
associated with the account you have requested we delete. If you used a phone
number to create your account, it cannot be used to create another account for
the same reasons. You will not be able to remove the primary email address for
the account or the phone number if you used it to create the account. We may
also be required to retain information to comply with a court settlement or
other equivalent mandated process.

Where we delete an account, we make our best efforts to delete all personal data
associated with your account. If we delete your personal data, we will both
render certain personal data about you permanently unrecoverable and also
deidentify certain personal data.

At all times, information collected by Apple will be treated in accordance with
Apple’s Privacy Policy, which can be found at

www.apple.com/privacy

Published Date: July 30, 2024

In [9]:
prompt = "What is the data retention policy in this privacy policy? " + policy

answer = prompt_llm(prompt = prompt, model_id = model_id)
pretty_print(answer)



According to the provided text, the data retention policy is as follows:

- When you make a purchase such as a subscription, Apple retains personal data
associated with your purchase for the periods specified by applicable laws
relating to financial reporting, which vary by region. For most customers, that
requires at least a 10-year retention period, but in regions such as China that
period can be 30 years. - Apple retains a record of your interactions with
iCloud-related services. These records are typically stored for 30 days but may
be stored for up to three years in an aggregated form, unless they are required
by law or legal process to store them for a longer period. - If you delete your
Apple ID account, Apple retains a record of the deleted account to comply with
legal obligations and for so long as necessary for the legitimate interests of
account security. However, Apple makes efforts to delete all personal data
associated with your account, rendering certain personal data about you
permanently unrecoverable and deidentifying certain personal data. - For iCloud
backup data, if you want to completely remove your backup data from iCloud, you
can delete your individual backups before turning off iCloud Backup. Otherwise,
the backup data will be retained for up to 180 days.

In [10]:
#let's help Li Wei

prompt = f"""Based on the data retention policy you described earlier:
{answer}
What is the specific retention period for purchase information and shipping addresses? Is there any mention of different retention periods for users in different countries?"""

response = prompt_llm(prompt=prompt, model_id=model_id)
pretty_print(response)



According to the data retention policy, the retention period for purchase
information is based on applicable laws relating to financial reporting. The
specific periods mentioned are:

- At least 10 years for most customers - Up to 30 years in regions such as China

This indicates that different retention periods may apply to users in different
countries or regions, with the example given being China having a retention
period of up to 30 years compared to at least 10 years for most other customers.

There is no specific mention of a retention period for shipping addresses, but
it is stated that the retention period for purchase information is tied to
applicable laws relating to financial reporting.

In [11]:
#Let's go more specific!

prompt = f"""Given the following information:
1. Li Wei made a purchase in 2010 (14 years ago)
2. Apple still retains detailed information about this purchase, including transaction details and shipping address
3. The data retention policy you described earlier: {answer}

Does it appear that Apple is complying with its own data retention policy in Li Wei's case? If not, what specific aspects of the policy might be violated?"""

response = prompt_llm(prompt=prompt, model_id=model_id)
pretty_print(response)



Based on the information provided, it appears that Apple is not complying with
its own data retention policy in Li Wei's case.

Specifically, the data retention policy for purchases states that Apple retains
personal data associated with a purchase for at least a 10-year retention period
for most customers. Since Li Wei made the purchase 14 years ago, which is beyond
the stated retention period of 10 years, Apple should have removed or de-
identified the detailed information about this purchase, including transaction
details and shipping address.

Apple's data retention policy does not provide any justification for retaining
detailed purchase information for more than 10 years. China's 30-year retention
period requirement does not seem to apply here since the question doesn't
explicitly mention Li Wei's location or any Chinese regulations applicable to
the case.

In [12]:
prompt = f"""Based on the privacy policy and the data retention information you provided earlier:
{answer}
What rights does Li Wei have regarding her data? Are there any specific procedures mentioned for requesting data deletion or challenging retention practices?"""

response = prompt_llm(prompt=prompt, model_id=model_id)
pretty_print(response)

Based on the provided text, Li Wei has the following rights regarding her data:

1. **Deletion of account and data**: Li Wei can delete her Apple ID account,
after which Apple will retain a record of the deleted account to comply with
legal obligations and for account security. However, Apple will make efforts to
delete personal data associated with the account, rendering it permanently
unrecoverable and deidentifying certain data. 2. **Deletion of backup data**: Li
Wei can delete her individual iCloud backups, and if she turns off iCloud
Backup, the backup data will be retained for up to 180 days.

Regarding specific procedures for requesting data deletion or challenging
retention practices, the provided text does not mention:

* A specific procedure for requesting data deletion, other than deleting
individual iCloud backups or deleting the entire Apple ID account. * Any
procedure for challenging retention practices or disputing data retention
periods.

It is likely that Li Wei would need to contact Apple's customer support or refer
to Apple's official website for more information on how to request data deletion
or challenge retention practices.

In [13]:
#Akira Tanaka - Cross-Border Data Transfer

prompt = f"""Referring back to the Apple privacy policy:
{policy}
What does the policy say about where user data is physically stored? Is there any mention of specific countries or regions where data centers are located?"""

response = prompt_llm(prompt=prompt, model_id=model_id)
pretty_print(response)

According to the Apple privacy policy, your data may be stored and processed in
both Apple and third-party data centers. The policy does not specify the exact
location of these data centers, however.

The policy mentions "both Apple and third-party data centers may be used to
store and process your data," and also notes that "your data may be accessed,
transferred to, and stored in servers within and outside your country of
residence, the servers being located in a country where Apple operates an
affiliate."

In [14]:
prompt = f"""Based on the Apple privacy policy:
{policy}
How does Apple handle cross-border data transfers? Are there any specific safeguards or considerations mentioned for users who access their data from different countries?"""

response = prompt_llm(prompt=prompt, model_id=model_id)
pretty_print(response)

The given Apple Privacy Policy does not explicitly mention cross-border data
transfers or country-specific safeguards. However, based on general information
and Apple's overall data protection policies, it can be inferred that Apple
complies with international data transfer regulations and standards for data
protection, such as:

1. **GDPR (General Data Protection Regulation)**: Apple has implemented
procedures to comply with GDPR for its users in the European Union, such as
ensuring their data is processed lawfully and has adequate security measures in
place. 2. **International data transfer frameworks**: Apple relies on data
transfer frameworks, such as the **Standard Contractual Clauses (SCCs)** for
transfers to the United States or other non-EU countries. Apple uses SCC-
approved adequacy frameworks for transferring personal data to the United States
from the EU and Switzerland. This provides assurance that the personal data
transferred outside the EU/Switzerland is protected to the same standard as in
the EU/Switzerland.

Apple also adheres to additional measures to ensure its users' data is protected
across countries, such as:

1. **Encryption**: Apple uses encryption to safeguard personal data when in
transit, regardless of the country from where data is accessed or provided. 2.
**Access controls**: Apple restricts access to personal data, with most Apple
team members having access only when it's required for performing their specific
job tasks.

However, it is essential for users to note that accessing data from different
countries might expose them to varying data protection standards and
jurisdictions. Apple disclaims responsibility for any issues that may arise due
to accessing their services from multiple countries.

While the exact language for cross-border data transfer in this policy is
missing, Apple emphasizes the importance of keeping an eye on their official
Privacy Policy page (www.apple.com/privacy) to get more details on their
international data transfer policies.

In [15]:
prompt = f"""Considering the Apple privacy policy and Akira's situation:
1. Akira is a Japanese citizen who frequently travels to the US
2. He uses iCloud+ to store work files, including confidential client information
3. He accesses this data from both Japan and the US

What specific privacy considerations or protections are relevant to Akira's case? Does the policy mention any differences in data handling or user rights for international users?"""

response = prompt_llm(prompt=prompt, model_id=model_id)
pretty_print(response)

Given Akira's situation, the following Apple privacy policy considerations are
relevant:

1. **Data Storage and Transfers**: As Akira stores data in iCloud+, Apple's data
storage and transfer policies apply. According to Apple, iCloud data is stored
in multiple locations, including the United States and other countries. This
means that Akira's data may be stored and transferred across borders.

*Protection*: Apple uses encryption to protect data in transit and at rest.
However, Akira should be aware that data may be subject to disclosure under the
laws of the countries where it is stored.

2. **Country-Specific Laws and Regulations**: As a Japanese citizen who accesses
data from both Japan and the US, Akira's data may be subject to different laws
and regulations in each country.

*Protection*: Apple notes that it complies with the Children's Online Privacy
Protection Act in the US and the General Data Protection Regulation (GDPR) in
the European Economic Area. However, for Japan, the policy mentions compliance
with the Act on the Protection of Personal Information. Apple's policy also
reserves the right to disclose data in response to government and court
requests.

3. **Data Request Policy**: This policy outlines how law enforcement,
governments, and users can request access to data. For non-US users like Akira,
Apple may respond to a request if the company is operating in the requested
country or if the data is in the country.

*Protection*: Apple has a data request policy in place to ensure that user data
is handled sensitively and in compliance with relevant laws. It also enables
users to request a copy of their stored data and permits users to delete their
iCloud data upon request.

4. **Japan-Specific Considerations**: In compliance with the Japanese Act on the
Protection of Personal Information, Apple allows users to:    - Request
disclosure, correction, suspension of use, or erasure of their personal data.
- Object to or restrict the use of their personal data.

*Protection*: Akira's rights as a Japanese citizen are safeguarded under this
Apple policy, ensuring he has greater control over his personal data stored and
processed by Apple.

Overall, Apple's policy provides international users with the same general
rights to data protection and control. However, Akira's right to data handling
might be influenced by different country-specific laws, and the extent of rights
regarding data handling across both Japan and the US will depend on compliance
with those local laws.

In [16]:
#chains

#One very important reasons to work with LLMs programtically has to do with CHAINING actions

#Let's say we are doing an in take system, where we have a system decide on how to route clients to lawyers

#For simplicity, let's say we want to know if the case is civil/criminal, and if it is civil, ask them if it is a high damages cases > $50k, and if it is criminal whether they have a police report.
#Then we take teh answer and draft an email to the lawyer.

client_input = """Yeah so I'm calling because I was trying to get a good lawyer. This neighbor of mine, you know,
he's been dumping his trash in my yard for months now. I've asked him to stop multiple times, but he just
laughs it off. Last week, he dumped a load of old furniture and it damaged my prized rosebushes.
I've had enough and want to sue him for the damage and to make him stop."""

#let's solve this problem together

In [17]:
#More models?

#Let's take "llava-v1.5-7b-4096-preview" for a spin.

model_id   = "llava-v1.5-7b-4096-preview"

#Now, check this image
url = "https://media.wired.com/photos/59a459d1e5238d59ce11404f/master/w_1920,c_limit/MemeLoveTriangle_297886754.jpg"

#What do we see here?

#Let's suppose we want to see if, on the day of the crime, there were two women caught by cctv
#can we automate something like this?



In [19]:
#we need a small function to take images from a url and translate them to the necessary format

def encode_image_from_url(image_url):
    response = requests.get(image_url)
    if response.status_code == 200:
        return base64.b64encode(response.content).decode('utf-8')
    else:
        raise Exception(f"Failed to retrieve image. Status code: {response.status_code}")

# Use the external image URL

prompt = "How many women are there in the image?"

response = prompt_llm(prompt=prompt, model_id = model_id, image_url=url)
print(response)

#this model isn't too great... there are better models

There are two women in the image.


OK, can we do sound?

Let's say your client reports of a heated debate she heard between a male and a female. There was possibly some reports of animal abuse, and you want to know whether that's true. But being a soft hearted person, you can't bring yourself to actually hear about that, so you'd rather let the machines do it for you.

https://github.com/yonathanarbel/AI-LAW/raw/ba7e96c49c18301944f21d77acaf4dcf19fa9852/Key%20moments%20from%20Donald%20Trump%20and%20Kamala%20Harris's%20US%20presidential%20debate%20BBCNews.mp3

In [20]:
model_id= "whisper-large-v3"

In [21]:

# Function to download the audio file locally
def get_audio(audio_url, save_path="downloaded_audio.mp3"):
    response = requests.get(audio_url)

    if response.status_code == 200:
        with open(save_path, 'wb') as audio_file:
            audio_file.write(response.content)
    else:
        raise Exception(f"Failed to retrieve audio. Status code: {response.status_code}")

# Updated raw audio URL from GitHub
audio_url = "https://github.com/yonathanarbel/AI-LAW/raw/ba7e96c49c18301944f21d77acaf4dcf19fa9852/Key%20moments%20from%20Donald%20Trump%20and%20Kamala%20Harris's%20US%20presidential%20debate%20BBCNews.mp3"

# Download the audio file locally
local_audio_path = "local_audio.mp3"
get_audio(audio_url, save_path=local_audio_path)

# Transcribe the downloaded audio file
prompt = "Listen to the recording and tell me if it mentions dogs or cats and the context"
response = prompt_llm(prompt=prompt, model_id="whisper-large-v3", audio_file_path=local_audio_path)




In [22]:
# Output the transcription or response
pretty_print(response)

Kamala Harris, it's a good debate. Good to see you. Have fun. Thank you. And I'm
going to actually do something really unusual, and I'm going to invite you to
attend one of Donald Trump's rallies, because it's a really interesting thing to
watch. And what you will also notice is that people start leaving his rallies
early out of exhaustion and boredom. First, let me respond to the rallies. She
said people start leaving. People don't go to her rallies. There's no reason to
go. And the people that do go, she's bussing them in and paying them to be
there. and then showing them in a different light. So she can't talk about that.
People don't leave my rallies. In Springfield, they're eating the dogs, the
people that came in. They're eating the cats. They're eating the pets of the
people that live there. ABC News did reach out to the city manager there. He
told us there had been no credible reports of specific claims of pets being
harmed, injured, or abused by individuals within the immigrant community. Well,
I've seen people on television. Let me just say here, this is the... The people
on television say my dog was taken and used for food. You talk about extreme.
They've destroyed the economy. And all you have to do is look at a poll. The
polls say 80 and 85 and even 90% that the Trump economy was great, that their
economy was terrible. Donald Trump has no plan for you. And when you look at his
economic plan, it's all about tax breaks for the richest people. I am offering
what I describe as an opportunity economy. She is Biden. You know, she's trying
to get away from Biden. I don't know the gentleman, she says. Clearly, I am not
Joe Biden. And I am certainly not Donald Trump.